In [1]:
import awkward as ak
import numpy as np
import pandas as pd
import numba
import glob
from tqdm import tqdm
print(ak.__version__)
from particletools.tables import PYTHIAParticleData as dataname

run_local = True
debug = True
if run_local:
    filepath = "/home/elias/taubus/V37nano/V37nano_VBFHToTauTau*"
else:
    filepath = "/eos/user/e/eleutgeb/menutools/Phase2-L1MenuTools/cache/V37nano/V37nano_VBFHToTauTau*"

debug = True

files = glob.glob(filepath)

#Get the files:
arrays = [ak.from_parquet(file) for file in files]
# Get the objectnames:
import re
if run_local:
    regex = re.compile(r'^{}|{}$'.format(re.escape("/home/elias/taubus/V37nano/V37nano_VBFHToTauTau_"), re.escape(".parquet")))
else: 
    regex = re.compile(r'^{}|{}$'.format(re.escape("/eos/user/e/eleutgeb/menutools/Phase2-L1MenuTools/cache/V37nano/V37nano_VBFHToTauTau_"), re.escape(".parquet")))


objectnames = [regex.sub("",file) for file in files]
print(objectnames)

2.6.4
['L1puppiJetHisto', 'L1gmtMuon', 'L1TrackMET', 'GenPart', 'GenVisTau', 'L1nnPuppiTau', 'L1tkElectron', 'L1EGendcap', 'L1gmtTkMuon', 'L1caloJet', 'L1hpsTau', 'L1puppiJetSC4', 'GenJet', 'L1caloTau', 'L1tkPhoton', 'L1EGbarrel', 'L1nnCaloTau', 'L1TrackHT', 'GenMET', 'L1TrackJet', 'GenJetAK8']


In [2]:
#Store objects in a dict
objectsdict = dict()
for idx,item in enumerate(objectnames): 
    objectsdict[item] = arrays[idx]

In [3]:
calotaus = objectsdict["L1caloTau"]
nncalotaus = objectsdict["L1nnCaloTau"]
nntaus = objectsdict["L1nnPuppiTau"]
jets = objectsdict["GenJet"]
taus = objectsdict["GenVisTau"]
l1jets = objectsdict["L1puppiJetHisto"]
puppijets = objectsdict["L1puppiJetHisto"]
elecsbarrel = objectsdict["L1EGbarrel"]
elecsendcap = objectsdict["L1EGendcap"]
elecsendcap_gt =  ak.pad_none(elecsendcap,6,axis=1,clip=True)
elecsbarrel_gt =  ak.pad_none(elecsbarrel,6,axis=1,clip=True)
muons = objectsdict["L1gmtMuon"]
tkmuons = objectsdict["L1gmtTkMuon"]
met =objectsdict["L1TrackMET"]
calojets = objectsdict["L1caloJet"]
tkele = objectsdict["L1tkElectron"]


In [4]:
calojets.fields
calotaus.fields

['L1caloTau_eta', 'L1caloTau_phi', 'L1caloTau_pt']

In [5]:
#helper functions:
import vector
vector.register_awkward()
x = objectsdict["L1EGbarrel"].fields


def prepare_objects(dictobj):
    fourvectordict = dict()
    standardizeddict = dict()
    vector.register_awkward()

    for key,value in dictobj.items():
        standardizeddict[key] = value
        fields = value.fields
        for field in fields:
            standardname = field.replace(key+"_","")
            standardizeddict[key][standardname] = dictobj[key][field]
            standardizeddict[key] = ak.without_field(standardizeddict[key],field)
            dictobj[key] = ak.without_field(value,field)
        print(key,":",standardizeddict[key].fields)
        fourvectordict[key] =  ak.Array(standardizeddict[key],with_name="Momentum3D")
    return fourvectordict,standardizeddict



    # # combining to numpy
def prepare_for_ml(objects,padlist,namelist,pad_val = 0):
    pad_arrs = []
    var_names = []
    for idx,object in enumerate(objects):
        topad = padlist[idx]
        name = namelist[idx]
        
        if topad == 1:
            pad_arr = object
        else:
            pad_arr = ak.pad_none(object,topad,axis=1,clip=True)
        for i in range(topad):
            for var in object.fields:
                if topad == 1:
                    pad_arrs += [ak.to_numpy(pad_arr[var][:])]
                else:
                    pad_arrs += [ak.to_numpy( ak.fill_none(pad_arr[var][:,i], pad_val) )]
                var_names.append( "{}_{}_{}".format(name, i, var) )
    return var_names, pad_arrs 




In [6]:
padlist = [1,1,5,5,5,5,5]
fourvectordict,standardizeddict = prepare_objects(objectsdict)
objects = [standardizeddict["L1TrackMET"],standardizeddict["L1TrackHT"]]
names = ["L1TrackMET","L1TrackHT"]
# del standardizeddict["L1EGbarrel"]["L1EGbarrel_eta"]
print(standardizeddict["L1EGbarrel"].fields)
standardizeddict["L1puppiJetHisto"] = ak.without_field(standardizeddict["L1puppiJetHisto"],'et')
standardizeddict["L1caloJet"] = ak.without_field(standardizeddict["L1caloJet"],'et')

inpnames,y = prepare_for_ml(objects,padlist,names)

usecalo = 1
if usecalo:
    v4l1jets = fourvectordict["L1caloJet"]
    v4nntaus = fourvectordict["L1caloTau"]
    l1jets = standardizeddict["L1caloJet"]
    nntaus = standardizeddict["L1caloTau"]
else: 
    v4l1jets = fourvectordict["L1puppiJetHisto"]
    l1jets = standardizeddict["L1puppiJetHisto"]
    v4nntaus = fourvectordict["L1nnPuppiTau"]
    nntaus = standardizeddict["L1nnPuppiTau"]



L1puppiJetHisto : ['et', 'eta', 'phi', 'pt']
L1gmtMuon : ['charge', 'chargeNoPh', 'hwBeta', 'hwEta', 'hwIso', 'hwPhi', 'hwPt', 'hwQual', 'd0', 'eta', 'phi', 'pt', 'z0']
L1TrackMET : ['pt']
GenPart : ['pt', 'eta', 'phi', 'pdgId', 'statusFlags']
GenVisTau : ['status', 'charge', 'genPartIdxMother', 'eta', 'mass', 'phi', 'pt']
L1nnPuppiTau : ['charge', 'id', 'passLooseNN', 'passLooseNNMass', 'passLoosePF', 'passMass', 'passTightNN', 'passTightNNMass', 'passTightPF', 'chargedIso', 'dXY', 'eta', 'fullIso', 'phi', 'pt', 'z0']
L1tkElectron : ['eleId', 'phoId', 'saId', 'hwEta', 'hwIso', 'hwPhi', 'hwPt', 'hwQual', 'charge', 'eta', 'phi', 'pt', 'relIso', 'z0']
L1EGendcap : ['eleId', 'phoId', 'saId', 'hwQual', 'eta', 'phi', 'pt']
L1gmtTkMuon : ['charge', 'chargeNoPh', 'hwBeta', 'hwEta', 'hwIso', 'hwPhi', 'hwPt', 'hwQual', 'd0', 'eta', 'phi', 'pt', 'z0']
L1caloJet : ['et', 'eta', 'phi', 'pt']
L1hpsTau : ['eta', 'phi', 'pt']
L1puppiJetSC4 : ['pt', 'eta', 'phi']
GenJet : ['pt', 'eta', 'phi', 'partonF

In [7]:
# x = nntaus[0][0]
# x.values()
objarray = np.vstack(y)
print(len(objarray[:,0]))
len(inpnames)
for idx,name in enumerate(inpnames):
    print(name,objarray[idx,0])
    
#delta arrays:

import vector
if debug:
    print(len(objarray[:,0]))

len(nntaus)


def objdeltas(obj1,obj2):
    obj1  = obj2



3
L1TrackMET_0_pt 35.84375
L1TrackHT_0_ht 83.5
L1TrackHT_0_mht 22.53125
3


In [8]:
ak.sum(standardizeddict["L1tkElectron"].pt)

2666872.2

In [9]:

fourvectordict["L1gmtMuon"].type.show()

99701 * var * Momentum3D[
    charge: int32,
    chargeNoPh: int32,
    hwBeta: int32,
    hwEta: int32,
    hwIso: int32,
    hwPhi: int32,
    hwPt: int32,
    hwQual: int32,
    d0: float32,
    eta: float32,
    phi: float32,
    pt: float32,
    z0: float32
]


In [10]:

def DeltaRcalc(o1,o2):
    #returns the delta in the dimensions of the second object with delta R  first objects, second
    combo = ak.cartesian({"offl": o1,"trgobj": o2}, nested=True ,axis=-1)
    off,trg = ak.unzip(combo)
    deltaR = off.deltaR(trg)
    return deltaR

def mapObjects(o1,o2):
    combo = ak.cartesian({"offl": o1,"trgobj": o2}, nested=True)
    off,trg = ak.unzip(combo)
    return off,trg


def sortbydeltaR(o1,o2):
    o1cart,o2cart = mapObjects(o1,o2)
    dR = DeltaRcalc(o1,o2)
    sorteddR = ak.argsort(dR)    
    return sorteddR

def dRtonumpy(o1,dr):
    o1dR = o1[dr]
    ttpad = ak.pad_none(o1dR,3,clip=True)
    nptt = ak.to_numpy(ttpad)
    npreg = stuctured_to_unstructured(nptt)
    return ak.fill_none(npreg)


    
test = fourvectordict["L1gmtMuon"]
deltartaumuons = DeltaRcalc(test,v4nntaus)

# Gen Taus vs NN Taus
combo = ak.cartesian({"offl": fourvectordict["GenVisTau"],"trgobj": fourvectordict["L1caloTau"]}, nested=True)
offmu,trgobj = ak.unzip(combo)
deltaRtaus_nnTaus = trgobj.deltaR(offmu) #magic?!

# L1Jets vs NN Taus
combo = ak.cartesian({"offl": v4l1jets,"trgobj": v4nntaus}, nested=True)
offmu,trgobj = ak.unzip(combo)
deltaRl1jets_nnTaus = offmu.deltaR(trgobj) #magic?!

combo = ak.cartesian({"trgobj": v4nntaus,"offl": v4l1jets}, nested=True)
offmu,trgobj = ak.unzip(combo)
deltaRnnTaus_l1jets = trgobj.deltaR(offmu) #magic?!



#map 
combo_pt =  ak.cartesian({"a": v4l1jets, "b":v4nntaus},nested=True)
trgjets,trgtaus = ak.unzip(combo_pt)

ptpercentage = trgjets.pt/trgtaus.pt



input_jets = []
input_taus = []
input_dRs = []
input_ptperc = []
input_is_true_tau = []
input_eles_barrel = []
input_eles_endcap = []

input_muons = []
input_false_eles = []




l1jets_near_nnTaus = (deltaRl1jets_nnTaus < 0.3)
nntaus_near_taus_old = ak.any((deltaRtaus_nnTaus < 0.3),axis=1)


In [11]:
#calculate DeltaR:
dr = DeltaRcalc(fourvectordict["GenVisTau"],fourvectordict["L1caloTau"])
# make flat pattern with empty fields, in this case where there are trigger taus and no gen taus:
drleng = ak.num(dr)
emptydr = drleng==0
#calculate the non matched taus,this is used only for the mask:
# Dr_greater = ak.any(dr >= 0.3,axis=-2,keepdims=True, mask_identity=False)  
Dr_smaller = ak.any(dr < 0.3,axis=-2,keepdims=True, mask_identity=False)  
Dr_greater = ~Dr_smaller

temptaus = ak.mask(fourvectordict["L1caloTau"],Dr_greater)
# print("near",v4truetaus[15])
temp_length = ak.num(temptaus,axis=2)
temp_mask = temp_length==0
Dr_greater_flat = ak.flatten(Dr_greater,axis=-1)
falsepattern = ak.where(emptydr,temp_mask,Dr_greater_flat)
nntaus_no_near_taus = falsepattern
nntaus_near_taus = ~ nntaus_no_near_taus
tt = ak.mask(standardizeddict["L1caloTau"],falsepattern)


# nntaus_near_taus = ak.any(dr < 3,axis=1,keepdims=True, mask_identity=False)  
# nntaus_no_near_taus = ak.any(dr >= 3,axis=1,keepdims=True, mask_identity=False)  

#we also have to check where there are no hadronic taus in the event
#done by counting dr values and then masking the list
# drleng = ak.num(dr)
# emptydr = ak.any(drleng == 0,axis=0, keepdims=True, mask_identity=False)
# nntaus_near_taus = ak.any((dr < 0.3),axis=1)
v4truetaus = ak.drop_none(ak.mask(fourvectordict["L1caloTau"],nntaus_near_taus),axis=1)
v4falsetaus =  ak.drop_none(ak.mask(fourvectordict["L1caloTau"],nntaus_no_near_taus),axis=1)
combo = ak.cartesian({"trgobj": v4truetaus,"offl": v4l1jets}, nested=True)
offmu,trgobj = ak.unzip(combo)
deltaRl1jets_nnTaus = offmu.deltaR(trgobj) #magic?!
truejets = l1jets[ak.argmin(deltaRl1jets_nnTaus,axis=-1)]
combo = ak.cartesian({"trgobj": v4falsetaus,"offl": v4l1jets}, nested=True)
offmu,trgobj = ak.unzip(combo)
deltaRl1jets_nnTaus = offmu.deltaR(trgobj) #magic?!
falsejets = l1jets[ak.argmin(deltaRl1jets_nnTaus,axis=-1)]
truetaus = ak.drop_none(ak.mask(standardizeddict["L1caloTau"],nntaus_near_taus),axis=1)
falsetaus = ak.drop_none(ak.mask(standardizeddict["L1caloTau"],nntaus_no_near_taus),axis=1)
print(v4falsetaus[15])
# test =  + ak.mask(standardizeddict["L1caloTau"],emptydr)  
# falsetaus = ak.concatenate(falsetaus,ak.drop_none(test,axis=-1),axis=1)


[{eta: 1.09, phi: 1.35, pt: 117}, {...}, {eta: -4.1, phi: -1.18, pt: 84}]


In [12]:
print(nntaus_near_taus_old[0])
Dr_smaller = ak.any(dr <= 0.3,axis=-2,keepdims=True, mask_identity=False)  
#[False, True, True, False, False, False]
print(Dr_smaller[0])



[False, False, False]
[[False, False, False]]


In [13]:

print(nntaus_near_taus[0])
# for i in range(1000):
#     if not ak.almost_equal(nntaus_near_taus[i], nntaus_near_taus_old[i]):
#         print(nntaus_near_taus[i])
#         print(nntaus_near_taus_old[i])
#         print(i)
# # v4truetaus[15]
# print(nntaus_near_taus_old[0])
# print(nntaus_near_taus[0])
# Dr_smaller_test = ak.any(dr < 3,axis=None,keepdims=True, mask_identity=False)  

# for item in standardizeddict["L1caloTau"][0]:
#     print(item)

# for item in standardizeddict["GenVisTau"][0]:
#     print(item['eta'])
#     print(item['phi'])
        
# print("dr",dr[0])
# print("drnew",Dr_smaller_test[0])
print(fourvectordict["L1caloTau"][1][1].deltaR(fourvectordict["GenVisTau"][1]))
x = ak.flatten(nntaus_near_taus,axis=None)
unique, counts = np.unique(x, return_counts=True)
print(dict(zip(unique, counts)))
print(len(x))

# print(len())
# # {0.0: 216168, 1.0: 88096}

# print(len(x))
# for i in range(50):
#     print(len(standardizeddict["L1caloTau"][i]))
#     print(len(Dr_greater_flat[i]))

ak.flatten(fourvectordict["L1caloTau"],axis=-1).type.show()

[False, False, False]
[0.0382, 1.21]
{False: 251829, True: 88096}
339925
339925 * Momentum3D[
    eta: float32,
    phi: float32,
    pt: float32
]


In [14]:
ttestt = ak.num(fourvectordict["L1caloTau"])
xxx = ak.unflatten(x,ttestt)
print(xxx[0])
xx = ak.with_field(fourvectordict["L1caloTau"],xxx,"test")
print(xx[0])



[False, False, False]
[{eta: -3.93, phi: 1.44, pt: 50.6, test: False}, {...}, {eta: -0.917, ...}]


In [15]:
# print(ak.any(fourvectordict["L1caloTau"][1][1].deltaR(fourvectordict["GenVisTau"][1]))

In [16]:

# muonstrue,b = ak.broadcast_arrays(standardizeddict["L1gmtMuon"],v4truetaus, depth_limit=1)
# v4muonstrue,b = ak.broadcast_arrays(fourvectordict["L1gmtMuon"],v4truetaus, depth_limit=2)
# test = ak.drop_none(v4muonstrue)
# drmu = DeltaRcalc(v4truetaus,fourvectordict["L1gmtMuon"])
# test = ak.argsort(drmu)
# test.type.show()
# drmu.type.show()

# b.type.show()
# x = fourvectordict["L1gmtMuon"]
# v4muonstrue[test]
# for l in range(len(drmu)):
#     tt = fourvectordict["L1gmtMuon"][l]
#     print(l)
#     tt[test[l]]
# fourvectordict["L1gmtMuon"][547]
# standardizeddict["L1gmtMuon"][ak.argsort(deltartaumuons,axis=1)]

def PrepareReldR(a,b,c,d,sort):
    f,g = mapObjects(a,b)
    drmu = DeltaRcalc(c,d)
    tts = ak.argsort(drmu)
    muonssorted = g[tts]
    # test = ak.any(nntaus_near_taus,axis=1)
    mm = ak.flatten(muonssorted[ak.any(sort,axis=1)],axis=1)
    return mm
# v4l1jets = fourvectordict["L1puppiJetHisto"]

#remove empty fields:
for key, value in standardizeddict.items():
    for field in value.fields:
        if ak.sum(value[field]) == 0:
            standardizeddict[key] = ak.without_field(standardizeddict[key],field)
#remove redundant information:
removal = ["L1gmtMuon",
"L1gmtTkMuon",
]            
for i in removal:
    print("before",standardizeddict[i].fields)
    standardizeddict[i] = ak.without_field(standardizeddict[i],'pt')
    standardizeddict[i] = ak.without_field(standardizeddict[i],'eta')
    standardizeddict[i] = ak.without_field(standardizeddict[i],'phi')
    print("after",standardizeddict[i].fields)


for key, value in standardizeddict.items():
    print(key,":",value.fields)
# truecalojets = PrepareReldR(truetaus,standardizeddict["L1caloJet"],v4truetaus,fourvectordict["L1caloJet"],nntaus_near_taus)
# falsecalojets = PrepareReldR(falsetaus,standardizeddict["L1caloJet"],v4falsetaus,fourvectordict["L1caloJet"],~nntaus_near_taus)
truemuons = PrepareReldR(truetaus,standardizeddict["L1gmtMuon"],v4truetaus,fourvectordict["L1gmtMuon"],nntaus_near_taus)
falsemuons = PrepareReldR(falsetaus,standardizeddict["L1gmtMuon"],v4falsetaus,fourvectordict["L1gmtMuon"],~nntaus_near_taus)
truetkmuons = PrepareReldR(truetaus,standardizeddict["L1gmtTkMuon"],v4truetaus,fourvectordict["L1gmtTkMuon"],nntaus_near_taus)
falsetkmuons = PrepareReldR(falsetaus,standardizeddict["L1gmtTkMuon"],v4falsetaus,fourvectordict["L1gmtTkMuon"],~nntaus_near_taus)
trueg = PrepareReldR(truetaus,standardizeddict["L1tkPhoton"],v4truetaus,fourvectordict["L1tkPhoton"],nntaus_near_taus)
falseg = PrepareReldR(falsetaus,standardizeddict["L1tkPhoton"],v4falsetaus,fourvectordict["L1tkPhoton"],~nntaus_near_taus)
trueeles = PrepareReldR(truetaus,standardizeddict["L1tkElectron"],v4truetaus,fourvectordict["L1tkElectron"],nntaus_near_taus)
falseeles = PrepareReldR(falsetaus,standardizeddict["L1tkElectron"],v4falsetaus,fourvectordict["L1tkElectron"],~nntaus_near_taus)
truej = PrepareReldR(truetaus,standardizeddict["L1puppiJetHisto"],v4truetaus,fourvectordict["L1puppiJetHisto"],nntaus_near_taus)
falsej = PrepareReldR(falsetaus,standardizeddict["L1puppiJetHisto"],v4falsetaus,fourvectordict["L1puppiJetHisto"],~nntaus_near_taus)
truecaloeb = PrepareReldR(truetaus,standardizeddict["L1EGbarrel"],v4truetaus,fourvectordict["L1EGbarrel"],nntaus_near_taus)
falsecaloeb = PrepareReldR(falsetaus,standardizeddict["L1EGbarrel"],v4falsetaus,fourvectordict["L1EGbarrel"],~nntaus_near_taus)
truecaloee = PrepareReldR(truetaus,standardizeddict["L1EGendcap"],v4truetaus,fourvectordict["L1EGendcap"],nntaus_near_taus)
falsecaloee = PrepareReldR(falsetaus,standardizeddict["L1EGendcap"],v4falsetaus,fourvectordict["L1EGendcap"],~nntaus_near_taus)
truecalojets = PrepareReldR(truetaus,standardizeddict["L1caloJet"],v4truetaus,fourvectordict["L1caloJet"],nntaus_near_taus)
falsecalojets = PrepareReldR(falsetaus,standardizeddict["L1caloJet"],v4falsetaus,fourvectordict["L1caloJet"],~nntaus_near_taus)
tobemapped = [truemuons,trueeles,trueg,truej, truetkmuons,truecalojets]
ls = truemuons.fields + trueeles.fields + trueg.fields + truej.fields + truetkmuons.fields + truecalojets.fields
names = ["L1gmtMuon",
"L1tkElectron",
"L1tkPhoton",
"L1puppiJetHisto",
"L1gmtTkMuon",
"L1caloJet"
]
#remove non physical vals:

padlist = [2,5,5,5,3,4]
drtruenames, drtruevals = prepare_for_ml(tobemapped,padlist,names)


tobemapped = [falsemuons,falseeles,falseg,falsej, falsetkmuons,falsecalojets]
for item in tobemapped:
    print(len(item))
ls = falsemuons.fields + falseeles.fields  + falseg.fields + falsej.fields + falsecaloeb.fields + falsecaloee.fields + falsetkmuons.fields
#padlist = [5,5,5,5]
drfnames, temp = prepare_for_ml(tobemapped,padlist,names)

drfvals = np.vstack(temp)
drtruevals = np.vstack(drtruevals)
drtruevals =np.swapaxes(drtruevals,0,1)
drfvals =np.swapaxes(drfvals,0,1)
drfvals = ak.to_numpy(drfvals)
drtruevals = ak.to_numpy(drtruevals)

from numpy.lib.recfunctions import structured_to_unstructured
# drfvals = structured_to_unstructured(drfvals)
# drtruevals = structured_to_unstructured(drtruevals)

# f,g = mapObjects(truetaus,standardizeddict["L1gmtMuon"])
# drmu = DeltaRcalc(v4truetaus,fourvectordict["L1gmtMuon"])
# # tt = f.deltaR(g)
# tts = ak.argsort(drmu)
# # bb,muons = ak.unzip(ak.cartesian([truetaus,standardizeddict["L1gmtMuon"]]))
# muonssorted = g[tts]
# # test = ak.any(nntaus_near_taus,axis=1)
# mm = ak.flatten(muonssorted[ak.any(nntaus_near_taus,axis=1)],axis=1)
# mm.type.show()
# ttflat = ak.flatten(truetaus)
# ttflat.type.show()


before ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'eta', 'phi', 'pt']
after ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual']
before ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'eta', 'phi', 'pt', 'z0']
after ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'z0']
L1puppiJetHisto : ['eta', 'phi', 'pt']
L1gmtMuon : ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual']
L1TrackMET : ['pt']
GenPart : ['pt', 'eta', 'phi', 'pdgId', 'statusFlags']
GenVisTau : ['status', 'charge', 'genPartIdxMother', 'eta', 'mass', 'phi', 'pt']
L1nnPuppiTau : ['id', 'passLooseNN', 'passLooseNNMass', 'passLoosePF', 'passMass', 'passTightNN', 'passTightNNMass', 'passTightPF', 'chargedIso', 'eta', 'phi', 'pt', 'z0']
L1tkElectron : ['eleId', 'phoId', 'saId', 'hwQual', 'charge', 'eta', 'phi', 'pt', 'relIso', 'z0']
L1EGendcap : ['eleId', 'saId', 'hwQual', 'eta', 'phi', 'pt']
L1gmtTkMuon : ['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'z0']
L1caloJet : ['eta', 'p

In [17]:
print(ls)
debug= 0
if(debug==1):
    for item in tobemapped:
        print(len(item))

['charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'eleId', 'phoId', 'saId', 'hwQual', 'charge', 'eta', 'phi', 'pt', 'relIso', 'z0', 'eleId', 'phoId', 'saId', 'hwQual', 'eta', 'phi', 'pt', 'relIso', 'eta', 'phi', 'pt', 'eleId', 'phoId', 'saId', 'hwQual', 'eta', 'phi', 'pt', 'eleId', 'saId', 'hwQual', 'eta', 'phi', 'pt', 'charge', 'hwBeta', 'hwEta', 'hwPhi', 'hwPt', 'hwQual', 'z0']


In [18]:
# # tt = fourvectordict["L1gmtMuon"][547]
# print(len(v4truetaus[15]))
# print(len(v4falsetaus[15]))
# print(len(standardizeddict["L1caloTau"][15]))
# print(len(fourvectordict["GenVisTau"][15]))
# nntaus_near_taus = (dr < 0.3)

# tt = ak.mask(standardizeddict["L1caloTau"],emptydr)
# print(emptydr)
# print(ak.is_none(tt[15]))
# print("tt",tt[15])
# tt2 = ak.is_none(tt,axis=-2)
# print(tt2[15])
# falsetaus = ak.mask(standardizeddict["L1caloTau"], dr > 3)
# print(falsetaus)

# print("dr",nntaus_near_taus[15])
# combo = ak.cartesian({"offl": fourvectordict["GenVisTau"],"trgobj": fourvectordict["L1caloTau"]}, nested=True)
# offmu,trgobj = ak.unzip(combo)
# deltaRtaus_nnTaus = offmu.deltaR(trgobj) 
# print(offmu[15])
# print(trgobj[15])
# deltaRtaus_nnTaus[15]


In [19]:
# drleng = ak.num(dr)
# print("len",drleng)
# emptydr = drleng==0
# print("e,",emptydr)
# nntaus_near_taus = ak.any(dr < 3,axis=-2,keepdims=True, mask_identity=False)  
# # v4truetaus = ak.mask(fourvectordict["L1caloTau"],nntaus_near_taus)
# print("near",v4truetaus[15])
# xx = ak.num(v4truetaus,axis=2)
# xy = xx==0
# nntaus_near_taus = ak.flatten(nntaus_near_taus,axis=-1)
# pattern = ak.where(emptydr,xy,nntaus_near_taus)
# print("pat",pattern)
# print(xy[15])
# nntaus_near_taus = ak.mask(dr, dr <= 3)  
# # v4truetaus = fourvectordict["L1caloTau"][nntaus_near_taus] 
# # testtau = ak.mask(standardizeddict["L1caloTau"][15],dr[15] <= 3)                                                                                  
# # print("testtau",testtau[0])   
# # patternflat = ak.flatten(pattern,axis=-1)
# tt = ak.mask(standardizeddict["L1caloTau"],pattern)
# print(standardizeddict["L1caloTau"][0][1])
# for i in range(10):
#     print(pattern[i])
#     print("testtau",(tt[i]))

# print("chcch",pattern[15])
# print("testtau",(tt[15]))      
# # print("testtau",(tt[0][0]))   

# testtau = ak.mask(standardizeddict["L1caloTau"],nntaus_near_taus) 

# # test = ak.concatenate([tt,testtau], axis=1)
# print("conc",ak.drop_none(test,axis=-1))
# print(testtau[0][0])
# print(standardizeddict["L1caloTau"][0][1])
# # ttest2 = ak.drop_none(test)
# print(ttest2)

In [20]:
debug = 0
if debug:
    for item,value in enumerate(standardizeddict["L1caloTau"]):
        if len(value) != len(v4truetaus[item]) + len(v4falsetaus[item]):
            print(item)




In [21]:
#prepare trues:
#broadcast other objects to the form of truetaus:
#first flip axis
objswapped =np.swapaxes(objarray,0,1)
objswappedak = ak.Array(objswapped)
#output arrays
tobj,b = ak.broadcast_arrays(objswappedak[:,np.newaxis],truetaus)
tobjflat =ak.flatten(tobj)
nptobjflat = ak.to_numpy(tobjflat)
#transform truetaus to numpy
ttflat = ak.flatten(truetaus)
npttflat = ak.to_numpy(ttflat)
tjflat = ak.flatten(truejets)
nptjflat = ak.to_numpy(tjflat)

from numpy.lib.recfunctions import structured_to_unstructured
unstructuredtruetaus = structured_to_unstructured(npttflat)
unstructuredtruejets = structured_to_unstructured(nptjflat)

trues = np.concatenate((unstructuredtruetaus,drtruevals),axis=1)





In [22]:
truejetsnames = ["calojets_" + i for i in truejets.fields]
truetausnames = ["calotaus_" + i for i in truetaus.fields]

names =  truetausnames  + drfnames
print(len(names))
print(names)

153
['calotaus_eta', 'calotaus_phi', 'calotaus_pt', 'L1gmtMuon_0_charge', 'L1gmtMuon_0_hwBeta', 'L1gmtMuon_0_hwEta', 'L1gmtMuon_0_hwPhi', 'L1gmtMuon_0_hwPt', 'L1gmtMuon_0_hwQual', 'L1gmtMuon_1_charge', 'L1gmtMuon_1_hwBeta', 'L1gmtMuon_1_hwEta', 'L1gmtMuon_1_hwPhi', 'L1gmtMuon_1_hwPt', 'L1gmtMuon_1_hwQual', 'L1tkElectron_0_eleId', 'L1tkElectron_0_phoId', 'L1tkElectron_0_saId', 'L1tkElectron_0_hwQual', 'L1tkElectron_0_charge', 'L1tkElectron_0_eta', 'L1tkElectron_0_phi', 'L1tkElectron_0_pt', 'L1tkElectron_0_relIso', 'L1tkElectron_0_z0', 'L1tkElectron_1_eleId', 'L1tkElectron_1_phoId', 'L1tkElectron_1_saId', 'L1tkElectron_1_hwQual', 'L1tkElectron_1_charge', 'L1tkElectron_1_eta', 'L1tkElectron_1_phi', 'L1tkElectron_1_pt', 'L1tkElectron_1_relIso', 'L1tkElectron_1_z0', 'L1tkElectron_2_eleId', 'L1tkElectron_2_phoId', 'L1tkElectron_2_saId', 'L1tkElectron_2_hwQual', 'L1tkElectron_2_charge', 'L1tkElectron_2_eta', 'L1tkElectron_2_phi', 'L1tkElectron_2_pt', 'L1tkElectron_2_relIso', 'L1tkElectron_2_z

In [23]:
#prepare trues:
#broadcast other objects to the form of truetaus:
#first flip axis
objswapped =np.swapaxes(objarray,0,1)
objswappedak = ak.Array(objswapped)
#output arrays
tobj,b = ak.broadcast_arrays(objswappedak[:,np.newaxis],falsetaus)
tobjflat =ak.flatten(tobj)
nptobjflat = ak.to_numpy(tobjflat)
#transform truetaus to numpy
ttflat = ak.flatten(falsetaus)
npttflat = ak.to_numpy(ttflat)
tjflat = ak.flatten(falsejets)
nptjflat = ak.to_numpy(tjflat)

from numpy.lib.recfunctions import structured_to_unstructured
unstructuredtruetaus = structured_to_unstructured(npttflat)
unstructuredtruejets = structured_to_unstructured(nptjflat)

falses = np.concatenate((unstructuredtruetaus,drfvals),axis=1)



In [24]:
indexlist = drfnames
print(falses[0][0])
print(standardizeddict["L1caloJet"].eta[0][0])
print(falses[0][1])
print(standardizeddict["L1caloJet"].phi[0][0])


print(trues[0][0])
print(standardizeddict["L1caloJet"].eta[1][1])
print(falses[0][1])
print(standardizeddict["L1caloJet"].phi[1])
idx = indexlist.index("L1gmtMuon_0_hwPt")

print(standardizeddict["L1gmtTkMuon"].hwPt[0])
tt = falses[0]

print(np.sum(trues[:,0]))
for i in range(len(trues[0])):
    if np.max(trues[:,i]) == np.min(trues[:,i]):
        print(names[i])
# tt.where(529)

-3.92578125
-3.9257812
1.43994140625
1.4399414
-0.392822265625
-0.39282227
1.43994140625
[1.09, -2.57, -1.53, 1.44, -2.84, -0.829]
[529, 324, 102]
-639.96533203125


In [25]:


# # # loop through all events
# # for ievt in tqdm(range(100)):
# #     cut_deltaRl1jets_nnTaus = l1jets_near_nnTaus[ievt]
    
# #     # loop through each l1jet near an nnTau in the event
# #     for ijet in np.where(ak.any(cut_deltaRl1jets_nnTaus,axis=1))[0]:
# #         # for each l1jet loop through the taus near it and save the pair with their dR and the Tau truth value
# #         for itau in np.where(cut_deltaRl1jets_nnTaus[ijet])[0]:

# #             input_jets.append( [l1jets[ievt][ijet]['pt'], l1jets[ievt][ijet]['eta'], l1jets[ievt][ijet]['phi']])
# #             input_taus.append( [
# #                                 nntaus[ievt][itau]['pt'], nntaus[ievt][itau]['eta'], nntaus[ievt][itau]['phi'],
# #                                 nntaus[ievt][itau]['fulliso'], nntaus[ievt][itau]['z0'], nntaus[ievt][itau]['passtightnn'],
# #                                 nntaus[ievt][itau]['chg'], nntaus[ievt][itau]['dxy'], nntaus[ievt][itau]['passloosenn']
# #                                ])
# #             input_dRs.append(deltaRl1jets_nnTaus[ievt][ijet][itau])
# #             input_ptperc.append(ptpercentage[ievt][ijet][itau])
            
# #             if len(nntaus_near_taus[ievt]):
# #                 input_is_true_tau.append(nntaus_near_taus[ievt][itau])
# #                 input_muons.append(muons[ievt])
# #                 input_eles_barrel.append(elecsbarrel_gt[ievt])
# #                 input_eles_endcap.append(elecsendcap_gt[ievt])

# #             else:
# #                 input_is_true_tau.append(False)
# #                 input_muons.append(muons[ievt])
# #                 input_eles_barrel.append(elecsbarrel_gt[ievt])
# #                 input_eles_endcap.append(elecsendcap_gt[ievt])
# true_objs = []
# true_taus = []
# true_jets = []
# false_taus = []
# false_objs = []
# false_jets = []
# dRtruejettaus = []
# dRfalsejettaus = []
# debug = 0
# varnamestaus = nntaus.fields
# varnamesjets = l1jets.fields
# #loop through all events:
# for ievt in tqdm(range(len(nntaus))):
# # for ievt in tqdm(range(5000)):

#         #loop through all taus:
#         for itau in range(len(nntaus[ievt])):
        
#             #check if there is a tau in the event:
#             if len(nntaus_near_taus[ievt]):
#                 # append only taus with a delta r smaller than 0.3:
#                 if nntaus_near_taus[ievt][itau]:
#                     true_taus.append( 
#                                     list(nntaus[ievt][itau].to_list().values())
#                                     )    
#                     true_objs.append(objarray[:,ievt]) 
#                     closestjet = np.argmin(deltaRnnTaus_l1jets[ievt][itau])
#                     dRtruejettaus.append(closestjet)
#                     true_jets.append( [l1jets[ievt][closestjet]['pt'], l1jets[ievt][closestjet]['eta'], l1jets[ievt][closestjet]['phi']])
#                 else:
#                 # tau delta r bigger:
#                     false_taus.append(
#                                     list(nntaus[ievt][itau].to_list().values())
#                                     )    
#                     false_objs.append(objarray[:,ievt]) 
#                     closestjet = np.argmin(deltaRnnTaus_l1jets[ievt][itau])
#                     dRfalsejettaus.append(closestjet)

#                     false_jets.append( [l1jets[ievt][closestjet]['pt'], l1jets[ievt][closestjet]['eta'], l1jets[ievt][closestjet]['phi']])
#             else:
#                 # no jet:

#                 false_taus.append(
#                                 list(nntaus[ievt][itau].to_list().values())
#                                 )    
#                 false_objs.append(objarray[:,ievt]) 
#                 closestjet = np.argmin(deltaRnnTaus_l1jets[ievt][itau])
#                 dRfalsejettaus.append(closestjet)
#                 if type(l1jets[ievt][closestjet]) is float:
#                     print('strange behaviour',ievt)
#                     false_jets.append([0,0,0])
#                 else:
#                     false_jets.append( [l1jets[ievt][closestjet]['pt'], l1jets[ievt][closestjet]['eta'], l1jets[ievt][closestjet]['phi']])

#                 if debug:
#                     print("NN tau:",nntaus[ievt])
#                     print("Gen Tau", deltaRtaus_nnTaus[ievt])




In [26]:
# debug =  1
# if debug:
#     print(v4l1jets[0].phi)
#     print(deltaRl1jets_nnTaus[0])
#     print(nntaus[0].L1nnPuppiTau_pt)
#     print(deltaRnnTaus_l1jets[0][0])
#     print(deltaRl1jets_nnTaus[0])
#     print(ptpercentage[0])
#     np.argmin(deltaRnnTaus_l1jets[0][0])
#     # show me the jet that has the smallest delta R to a tau:
#     # for this we select the first tau and print the jet that has the smallest delta r

#     print(nntaus_near_taus[1])

In [27]:
# print(false_jets[0])
# print(false_jets[65281])
# false_jets[65281]
# print(ak.is_none(x))


In [28]:
print(l1jets[3404])

[{eta: -2.39, phi: -0.829, pt: 69.8}, {...}, {eta: 2.13, phi: -0.131, ...}]


In [29]:

# #convert to array:


# #temp workaround: i think there was no jet:
# # false_jets[3403] = [0.0,0.0,0.0]
# false_jets[64710] = [0.0,0.0,0.0]


# false_taus_arr = np.vstack(false_taus)
# false_jets_arr = np.vstack(false_jets)
# false_objs_arr = np.vstack(false_objs)
# false_drjettau_arr = np.array(dRfalsejettaus)
# false_drjettau_arr = false_drjettau_arr.reshape(len(false_drjettau_arr),1)

# true_taus_arr = np.vstack(true_taus)
# true_jets_arr = np.vstack(true_jets)
# true_objs_arr = np.vstack(true_objs)
# true_drjettau_arr = np.array(dRtruejettaus)
# true_drjettau_arr = true_drjettau_arr.reshape(len(true_drjettau_arr),1)

# if debug:
#     print("false objects dimensions:")
#     print("taus: ",false_taus_arr.shape)
#     print("jets: ",false_jets_arr.shape)
#     print("objs: ",false_objs_arr.shape)
#     print("drjt: ",false_drjettau_arr.shape)
#     print("true objects dimensions:")
#     print("taus: ",true_taus_arr.shape)
#     print("jets: ",true_jets_arr.shape)
#     print("objs: ",true_objs_arr.shape)
#     print("drjt: ",true_drjettau_arr.shape)



# x_bkg = np.concatenate((false_taus_arr,false_jets_arr,false_objs_arr,false_drjettau_arr),axis=1)
# x_sig = np.concatenate((true_taus_arr,true_jets_arr,true_objs_arr,true_drjettau_arr),axis=1)
# featurenames = varnamestaus + varnamesjets + inpnames + ["dr_jet_tau"]

# featurenames




In [30]:
x_sig = trues
x_bkg = falses


In [31]:

indexlist = drfnames
print(falses[0][0])
print(standardizeddict["L1caloJet"].eta[0][0])
print(falses[0][1])
print(standardizeddict["L1caloJet"].phi[0][0])

print(drfnames)
print(trues[0][0])
print(standardizeddict["L1caloJet"].eta[1][1])
print(falses[0][1])
print(standardizeddict["L1caloJet"].phi[1])
idx = indexlist.index("L1gmtTkMuon_0_hwPt")

print(standardizeddict["L1gmtTkMuon"].hwPt[0])

tt = falses[0]
print(ls)
# print(drfvals[0])
print(idx)
idx = indexlist.index("L1gmtTkMuon_1_hwPt")
print(idx)
print(len(drfvals[0]))
print(np.where(drfvals[0] == 529))
print(np.where(drfvals[0] == 324))
print(np.where(drfvals[0] == 102))

-3.92578125
-3.9257812
1.43994140625
1.4399414
['L1gmtMuon_0_charge', 'L1gmtMuon_0_hwBeta', 'L1gmtMuon_0_hwEta', 'L1gmtMuon_0_hwPhi', 'L1gmtMuon_0_hwPt', 'L1gmtMuon_0_hwQual', 'L1gmtMuon_1_charge', 'L1gmtMuon_1_hwBeta', 'L1gmtMuon_1_hwEta', 'L1gmtMuon_1_hwPhi', 'L1gmtMuon_1_hwPt', 'L1gmtMuon_1_hwQual', 'L1tkElectron_0_eleId', 'L1tkElectron_0_phoId', 'L1tkElectron_0_saId', 'L1tkElectron_0_hwQual', 'L1tkElectron_0_charge', 'L1tkElectron_0_eta', 'L1tkElectron_0_phi', 'L1tkElectron_0_pt', 'L1tkElectron_0_relIso', 'L1tkElectron_0_z0', 'L1tkElectron_1_eleId', 'L1tkElectron_1_phoId', 'L1tkElectron_1_saId', 'L1tkElectron_1_hwQual', 'L1tkElectron_1_charge', 'L1tkElectron_1_eta', 'L1tkElectron_1_phi', 'L1tkElectron_1_pt', 'L1tkElectron_1_relIso', 'L1tkElectron_1_z0', 'L1tkElectron_2_eleId', 'L1tkElectron_2_phoId', 'L1tkElectron_2_saId', 'L1tkElectron_2_hwQual', 'L1tkElectron_2_charge', 'L1tkElectron_2_eta', 'L1tkElectron_2_phi', 'L1tkElectron_2_pt', 'L1tkElectron_2_relIso', 'L1tkElectron_2_z0', 

In [32]:
print(len(trues))
print(len(falses))
trues[0].shape

88096
251829


(153,)

In [33]:

 # creating labels
y_bkg = np.zeros(len(x_bkg))
y_sig = np.ones(len(x_sig))

# combining signal & bkg
x = np.concatenate((x_bkg, x_sig))
y = np.concatenate((y_bkg, y_sig))
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# _ = scaler.fit(x)
# x_scaled = scaler.transform(x)


# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# pca = PCA(n_components=2)  # You can choose the number of components
# X_pca = pca.fit_transform(x_scaled)

# # Step 5: Analyze the results
# # Explained variance
# explained_variance = pca.explained_variance_ratio_
# print('Explained variance by each component:', explained_variance)

# # Plotting the principal components, colored by target
# plt.figure(figsize=(8, 6))
# scatter = plt.scatter(X_pca[:, 1], X_pca[:, 2], c=y, cmap='viridis', edgecolor='k', s=50)
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.title('PCA of Dataset')
# plt.colorbar(scatter)
# plt.grid(True)
# plt.show()

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
_ = scaler.fit(x)
x_scaled = scaler.transform(x)

In [35]:
 # make directory
import os
import pickle
from datetime import datetime

datenow = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
outdir = "smallnet_outputs_for_training_{}/".format(datenow)
if not os.path.exists(outdir):
    os.makedirs(outdir)

with open(outdir+"scaler.pkl", 'wb') as file_pi: 
   pickle.dump(scaler, file_pi)

print("outdir:  ",outdir)

outdir:   smallnet_outputs_for_training_2024_08_24-10_36_14_AM/


In [36]:

with open(outdir+"featurenames.pkl", 'wb') as file_pi: 
   pickle.dump(names, file_pi)

In [ ]:
# print(names)
# with open(outdir+"names.pkl", 'wb') as file_pi: 
#    pickle.dump(names, file_pi)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.5)



In [ ]:
# ak.to_parquet(X_train, outdir+"X_train_scaled.parquet")
# ak.to_parquet(y_train, outdir+"y_train_scaled.parquet")
# ak.to_parquet(X_test, outdir+"X_test_scaled.parquet")
# ak.to_parquet(y_test, outdir+"y_test_scaled.parquet")# 

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.4)
print(X_train.shape[1])
from sklearn.utils.class_weight import compute_class_weight

#weighing trues and false
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)

# Convert class_weights to a dictionary
class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict)


153
{0: 0.6760281873143827, 1: 1.9202270887077033}


In [38]:
ak.to_parquet(X_train, outdir+"X_train_scaled.parquet")
ak.to_parquet(y_train, outdir+"y_train_scaled.parquet")
ak.to_parquet(X_test, outdir+"X_test_scaled.parquet")
ak.to_parquet(y_test, outdir+"y_test_scaled.parquet")


  created_by: parquet-cpp-arrow version 16.0.0
  num_columns: 1
  num_rows: 135970
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [ ]:
truetaus.fields
truetaus_plot =ak.flatten(truetaus,axis=-1)
truetaus_plot.type.show()

falsetaus_plot =ak.flatten(falsetaus,axis=-1)


In [ ]:
import matplotlib.pyplot as plt


for i in truetaus.fields:
    # if "_0_" not in name: continue
        
    plt.figure()
    
    _ = plt.hist(falsetaus_plot[i], bins = 100, log = True, density = False, label = "Bkg")
    _ = plt.hist(truetaus_plot[i], bins = _[1], histtype = "step", density = False, label = "Sig")
    
    plt.xlabel(i)
    plt.legend()
    plt.show()

In [ ]:
def Preparemindrplot(a,b,c,d,sort):
    f,g = mapObjects(a,b)
    drmu = DeltaRcalc(c,d)
    tts = ak.min(drmu,axis=1)
    return ak.flatten(tts,axis=None)
    # muonssorted = g[tts]
    # test = ak.any(nntaus_near_taus,axis=1)
    # mm = ak.flatten(muonssorted[ak.any(sort,axis=1)],axis=1)

y = [
 "L1gmtMuon",
"L1gmtTkMuon",
"L1tkPhoton",
"L1tkElectron",
"L1puppiJetHisto",
"L1EGbarrel",
"L1EGendcap"]
for x in y:
    # standardizeddict[x] = ak.without_field(standardizeddict[x],['eta'])
    print(standardizeddict[x].fields)
    truemuons_plot =  Preparemindrplot(truetaus,standardizeddict[x],v4truetaus,fourvectordict[x],nntaus_near_taus)
    falsemuons_plot = Preparemindrplot(falsetaus,standardizeddict[x],v4falsetaus,fourvectordict[x],~nntaus_near_taus)
    # standardizeddict["L1gmtTkMuon"]
    print(ak.flatten(truemuons_plot,axis=None))

    plt.figure()
        
    _ = plt.hist(ak.flatten(truemuons_plot,axis=None), bins = 50, log = False, density = True, label = "Sig")
    _ = plt.hist(ak.flatten(falsemuons_plot,axis=None), bins = _[1], histtype = "step", density = True, label = "Bkg")
        
    plt.xlabel("dR")
    plt.ylabel("Counts")
    plt.title(x)
    plt.legend()
    plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import Dropout
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dropout(0.2))
model.add(Dense(64, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dropout(0.2))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(1, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='sigmoid', name='sigmoid'))

adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss=['binary_crossentropy'], metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
model, to_file='{}model.png'.format(outdir), show_shapes=True, show_dtype=True,
show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)


In [ ]:
%%time
history = model.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=10,
    class_weight=class_weight_dict,
    validation_split=0.2,
    shuffle=True,
    # callbacks=callbacks.callbacks,
)




In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
def plotTrainingHistory(history, metrics = ["loss", "accuracy"], f = None, axs = None):
    
    # creating the plot
    if not f and not axs: 
        f, axs = plt.subplots(len(metrics), 1, figsize = (12, 4*len(metrics)), sharex = True)
    if len(metrics) == 1:
        axs = [axs]
    plt.subplots_adjust(wspace=0, hspace=0)

    # labeling
#     hep.cms.label("private work", data=False, ax=axs[0])

    for i in range(len(metrics)):
        
        metric = metrics[i]
        ax = axs[i]
        ax.set_ylabel(metric)
        
        if isinstance(history, list): # handle kfold
            for foldi in range(len(history)):
                ax.plot(history[foldi].history[metric], color = "C{}".format(foldi))
                ax.plot(history[foldi].history['val_' + metric], color = "C{}".format(foldi), linestyle = "--")
                
            la2, = ax.plot([0,0], [0,0], color="Grey")
            lb2, = ax.plot([0,0], [0,0], color="Grey", linestyle = "--")
            ax.legend([la2, lb2], ["training", "validation"])
        else: 
            xs = np.arange(len(history.history['val_' + metric]))
            ax.plot(xs,history.history[metric], label = 'training')
            ax.plot(xs+.5, history.history['val_' + metric], label= 'validation')
            ax.legend()

    axs[-1].set_xlabel("Epoch")
    
    return f, axs


model.save(outdir + "/model.h5")

with open(outdir + "/history.pkl", 'wb') as file_pi: pickle.dump(history, file_pi)

fig = plotTrainingHistory(history)



In [ ]:
y_test_pred = model.predict(X_test)  # Save predictions:
ak.to_parquet(y_test_pred, outdir+"y_test_pred.parquet")  
from sklearn.metrics import roc_curve

fpr, tpr, thr = roc_curve(y_test, y_test_pred, drop_intermediate=False)

In [ ]:
 ## for L1 rate estimates from ZeroBias/SingleNuMC
def totalMinBiasRate(nCollBunch = 2500):
    LHCfreq = 11245.6
    return LHCfreq * nCollBunch / 1e3 # in kHz


plt.plot(fpr * totalMinBiasRate(), tpr)
plt.xlabel("FPR*MBrate: Trigger rate [kHz]")
plt.ylabel("TPR: signal efficiency")
plt.grid()
plt.xlim(0,1000)

In [ ]:
%time
y_test_pred = model.predict(X_test)  # Save predictions:
ak.to_parquet(y_test_pred, outdir+"y_test_pred.parquet")  

from sklearn.metrics import roc_curve

fpr, tpr, thr = roc_curve(y_test, y_test_pred, drop_intermediate=False)  
plt.plot(fpr, tpr)
plt.xlabel("FPR: background efficiency")
plt.ylabel("TPR: signal efficiency")
plt.grid()
plt.savefig(outdir+'efficiency.png')

In [ ]:
## for L1 rate estimates from ZeroBias/SingleNuMC
def totalMinBiasRate(nCollBunch = 2500):
    LHCfreq = 11245.6
    return LHCfreq * nCollBunch / 1e3 # in kHz  Let's now make the ROC plot and focus on a rate range 0-100 kHz: plt.plot(fpr * totalMinBiasRate(), tpr)
plt.plot(fpr * totalMinBiasRate(), tpr)
plt.xlabel("FPR*MBrate: Trigger rate [kHz]")
plt.ylabel("TPR: signal efficiency")
plt.grid()
plt.xlim(0,1000)
plt.savefig(outdir+'atrate.png')

In [ ]:
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import F1Score

# Define the model-building function
def build_model(hp):
    model = Sequential()
    
    # First Dense Layer
    model.add(Dense(
        units=hp.Int('units_fc1', min_value=128, max_value=512, step=64),
        input_shape=(X_train.shape[1],),
        name='fc1',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(hp.Float('reg1', 0.0001, 0.001, step=0.0001))
    ))
    model.add(Activation(activation='relu', name='relu1'))
    model.add(Dropout(hp.Float('dropout_fc1', 0.2, 0.5, step=0.1)))
    
    # Second Dense Layer
    model.add(Dense(
        units=hp.Int('units_fc2', min_value=32, max_value=128, step=32),
        name='fc2',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(hp.Float('reg2', 0.0001, 0.001, step=0.0001))
    ))
    model.add(Activation(activation='relu', name='relu2'))
    model.add(Dropout(hp.Float('dropout_fc2', 0.2, 0.5, step=0.1)))
    
    # Third Dense Layer
    model.add(Dense(
        units=hp.Int('units_fc3', min_value=16, max_value=128, step=16),
        name='fc3',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(hp.Float('reg3', 0.0001, 0.0004, step=0.0001))
    ))
    model.add(Activation(activation='relu', name='relu3'))
    
    # Output Layer
    model.add(Dense(1, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
    model.add(Activation(activation='sigmoid', name='sigmoid'))
    
    hp_learning_rate = hp.Choice('learning_rate', 
                                 values=[1e-2, 1e-3, 1e-4]
                                 )
    optimizers_dict = {
        "Adam":    Adam(learning_rate=hp_learning_rate),
        "SGD":     SGD(learning_rate=hp_learning_rate),
        "Adagrad": Adagrad(learning_rate=hp_learning_rate)
        }

    hp_optimizers = hp.Choice(
        'optimizer', 
        values=["Adam"]
        )

    model.compile(
        optimizer=optimizers_dict[hp_optimizers],
        loss=['binary_crossentropy'], metrics=['accuracy']
        )
    return model


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.regularizers import l1
from keras_tuner.tuners import RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=2,
    directory='opt9',
    project_name='withmorevars',

)

# Load your dataset here
# X_train, y_train, X_val, y_val = ...

# Define the search space for algorithm parameters
def hyperparameters(hp):
    return {
        'batch_size': hp.Int('batch_size', min_value=16, max_value=128, step=16),
        'learning_rate': hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log'),
    }

# Run the hyperparameter search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test), 
                 class_weight=class_weight_dict,
    validation_split=0.2,
             callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
])

# Retrieve and print the best hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units_fc1')}, the second layer is {best_hps.get('units_fc2')}, 
# the third layer is {best_hps.get('units_fc3')}, 
# the optimal dropout rate for the first layer is {best_hps.get('dropout_fc1')},
# the second layer is {best_hps.get('dropout_fc2')},
# the best optimizer is {best_hps.get('optimizer')},
# the best batch size is {best_hps.get('batch_size')},
# and the best learning rate is {best_hps.get('learning_rate')}.
# """)

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_fc1')}, the second layer is {best_hps.get('units_fc2')}, 
the third layer is {best_hps.get('units_fc3')}, 
the optimal dropout rate for the first layer is {best_hps.get('dropout_fc1')},
the second layer is {best_hps.get('dropout_fc2')},
the best optimizer is {best_hps.get('optimizer')},
and the best learning rate is {best_hps.get('learning_rate')}.
""")

In [ ]:

# Instantiate the tuner
# tuner = RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=10,
#     executions_per_trial=1,
#     directory='test9',
#     project_name='tuning_demo'
# )

# # Load your dataset here
# # X_train, y_train, X_val, y_val = ...

# # Start the hyperparameter search
# tuner.search(X_train, y_train, epochs=40, validation_data=(X_test, y_test))

# # Get the best hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=4)[0]

In [ ]:
# print(best_hps())
print("Best hyperparameters:")
print(f"units_fc1: {best_hps.get('units_fc1')}")
print(f"units_fc2: {best_hps.get('units_fc2')}")
print(f"units_fc3: {best_hps.get('units_fc3')}")
print(f"dropout_fc1: {best_hps.get('dropout_fc1')}")
print(f"dropout_fc2: {best_hps.get('dropout_fc2')}")
print(f"optimizer: {best_hps.get('optimizer')}")

In [ ]:
model = tuner.hypermodel.build(best_hps)
# history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
tensorboard_callback = tf.keras.callbacks.TensorBoard(outdir, histogram_freq=1)
print(outdir)


In [ ]:
model = tuner.hypermodel.build(best_hps)
model.save(filepath=outdir+"model")
%time
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=1000,
    class_weight=class_weight_dict,
    validation_split=0.2,
    shuffle=True,
    callbacks=tensorboard_callback,
)


In [ ]:


import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=featurenames, class_names=[0, 1], mode='classification')
exp = explainer.explain_instance(X_test[5], model.predict, num_features=20, top_labels=1)

In [ ]:
exp.available_labels()
explainer.feature_names

In [ ]:
exp.as_pyplot_figure(label=0)

In [ ]:
featurenames

In [ ]:
X_test.shape

In [ ]:
# import shap
# def predict_function(x):
#     # Assuming your Keras model takes input x and returns predictions
#     return model.predict(x)

# # Step 5: Generate SHAP explanations
# # Create a DeepExplainer object by passing the Keras model and the training data
# explainer = shap.DeepExplainer(model, X_train)

# # Explain predictions on the first instance from the test set
# shap_values = explainer.shap_values(X_test)

# # Step 6: Visualize feature importance
# shap.initjs()  # Initialize JavaScript for visualization (for Jupyter Notebook)
# shap.force_plot(explainer.expected_value[0], shap_values[0], X_test[0])